In [1]:
from prody import *
import numpy as np
from os.path import basename
import fnmatch
import os
import networkx as nx
from networkx.algorithms import community
from itertools import chain, combinations

# Community composition analysis (i) between segments and (ii) residue by residue. RGB coloring scheme for _$\Omega$_ = 6

### __Beware that your pdb and dcd should have the same name and you can write a string pattern for batch calculation__

In the next cell, please write start, stop and stride of the dcd file that you want to analyze.
Point out the maximum community size you want to write to output files

In [2]:
starting_frame = 110 # write starting frame of your dcd
stopping_frame = 210 # last frame of your dcd
stride = 10 # stride of your dcd file
max_community_size = 6 #maximum number of communities/omega

Write the file string pattern or your pdb file. Please realize that, that file name also will be used for parsing related dcd file. Further, please select atoms you want to coarse-grain and analyze as nodes.

In [9]:
file_string_pattern = '*tp*.pdb' #if you write *.pdb, it will go through all the pdb's in the 
                                        #working directory and use the dcd files with the same name
ligand_name = "TMP"
atom_selection_string = "(name CB and protein) or (name CA and resname GLY) or (name C2 and resname "+ligand_name+")"

#for DHFR we coarse grainded the ligands, for vanilla carbon beta you can comment out the following line
#atom_selection_string = "(name CB and protein) or (name CA and resname GLY)

There are three different calculation scheme for dynamic community composition.

**1) Community sharing segments over time.**
This calculates the community sharing fraction between two pre-assigned structural segments based on their residue numbers. By using this calculation, the user can understand the underlying changes over the commumity composition through the MD trajectory, and the user can detect the number of community seperation that is needed to seperate two segments. This is important since some of the effects may cancel on average.

**2) Average residue by residue community co-occupancy.**

Here, for each $\Omega$, the user can calculate the fraction of community sharing of each residue on time average. By using this information, the user can detect the group of residues that tend to be in the same community throught the MD trajectory.

**3) Protein structure coloring by RGB scheme that calculated by community sharing of reference residues.**

By deciding, three reference residues for red, green and blue, the user can calculate the fraction of community sharing belonging to other nodes based on the color code of reference nodes. This process may be done for certain $\Omega$, and the default is $\Omega$ = 6.

Please assign, True or False, for each calculation to be done.

In [10]:
#Between the segments

segment_calculation = False

segment_1 = list(range(60, 75)) 
segment_2 = [160, 161, 162, 163, 164, 165, 166, 167, 168, 169]

#Res by Res calculation 
res_by_res_calculation = False

residues = list(range(1, 160))
residues.extend(list(range(160, 162)))
residue = np.array(residues)

#RGS scheme coloring matrix
rgb_scheme = True
rgb_community = 6
red = [94]
green = [142]
blue = [64]

In [11]:
for file in os.listdir('.'):
    if fnmatch.fnmatch(file, file_string_pattern): 
        #Find the pdb and parse it
        pdb_name = file
        #print(pdb_name)
        structure = parsePDB(str(pdb_name))
        file_name_wh_ex = str(os.path.splitext(pdb_name)[0])
        structure_beta = structure.select(atom_selection_string)
        #DHFR corrections
        DHF = structure_beta.select("resname "+ligand_name)
        DHF.setResnums(list(range(160, 161)))
        #NADH = structure_beta.select("resname NDPH")
        #NADH.setResnums(list(range(160, 166)))
        #residue_number = len(structure_beta)
        residue_number = len(structure_beta)
        conf_size = len(range(starting_frame, stopping_frame, stride))*2
        #Initiate variables
        BC_data_matrix = np.zeros((residue_number, conf_size))
        comm_mat_btw_segment = np.zeros((max_community_size-1,conf_size))
        community_all_data_matrix = np.zeros((residue_number, residue_number, conf_size, max_community_size-1))
        rgb_matrix = np.zeros((residue_number, conf_size, 3))
        #Parse the dcd file
        for frame_interval in [[starting_frame,stopping_frame]]:
            dcd_1 = parseDCD(str(file_name_wh_ex)+".dcd",
                                      frame_interval[0],frame_interval[1],stride);
            dcd_2 = parseDCD(str(file_name_wh_ex)+"_2.dcd",
                                      frame_interval[0],frame_interval[1],stride);
            ensemble_cov_2 = dcd_1 + dcd_2
            print(ensemble_cov_2)
            
            #select the nodes from the protein structure
            res_index = structure_beta.getResnums()
            print("Here is the residue index of the nodes")
            print(' '.join(map(str, res_index)))
            print("\n")
            #bind the dcd and pdb files
            ensemble_cov_2_mean = (ensemble_cov_2.getCoordsets()).mean(0)
            ensemble_cov_2.setAtoms(structure_beta);
            ensemble_cov_2.setCoords(ensemble_cov_2_mean);
            #start the loop for the frames in dcd
            for conformation_num in range(len(ensemble_cov_2)):
                betas_gly_alphas = ensemble_cov_2[conformation_num].getAtoms()
                betas_gly_alphas_coords = ensemble_cov_2[conformation_num].getCoords()
                nodes = betas_gly_alphas
                nodes_range = len(nodes)
                nodes_list = nodes.getResnums()
#                print(nodes_list)
                print("Index of MD-simulation frame "+str(conformation_num))
    
                #Contruct the network, please locate the cut-off value
                ia_list = []
                for i in range(nodes_range-1):
                    for j in range(i+1, nodes_range):
                        dist = calcDistance(betas_gly_alphas_coords[i], betas_gly_alphas_coords[j])
                        if dist > 6.7:
                            continue
                        ia_list.append((nodes[i].getResnum(), nodes[j].getResnum()))
    
                protein_graph = nx.Graph()
                protein_graph.add_nodes_from(nodes_list)
                protein_graph.add_edges_from(ia_list)
            ############################################
                #### Calculate the node betweenness centrality
                betwen_cent = nx.betweenness_centrality(protein_graph, normalized=True)
                betwen_cent = np.asarray(list(betwen_cent.values()))
                betwen_cent = betwen_cent.reshape((residue_number,1))
                BC_data_matrix[:,conformation_num] = betwen_cent[:,0]
    
            ###########################################
            ###Calculate Girwan-Newman communities
                partition_girvan_newman = list(community.girvan_newman(protein_graph))
                partition_girvan_newman = partition_girvan_newman[0:max_community_size-1]
            ########## SEG by SEG community composition calculation
                if segment_calculation is True:
                    for i in range(max_community_size-1):
                        for comm in partition_girvan_newman[i]:
                            comm = list(comm)
                            if ((any(list(item in segment_1 for item in comm)) and any(list(item in segment_2 for item in comm)))) == True:
                                comm_mat_btw_segment[i,conformation_num] = 1
                                
            
            ####### RES BY RES community composition calculation
                if res_by_res_calculation is True:
                    for i in range(residue_number):
                        for j in range(residue_number):
                            for level in partition_girvan_newman:
                                for comm in level:
                                    comm = list(comm)
                                    if (residues[i] in comm) and (residues[j] in comm) == True:
                                        community_all_data_matrix[i, j, conformation_num, partition_girvan_newman.index(level)] = 1
            #### RGB scheme calculation                        
                if rgb_scheme is True:
                    for i in range(residue_number):
                        for comm in partition_girvan_newman[rgb_community-2]:
                            comm = list(comm)
                            if (red in comm):
                                is_in = np.isin(residues, comm)
                                rgb_matrix[np.where(is_in)[0], conformation_num, 0] = 1
                            if (green in comm):
                                is_in = np.isin(residues, comm)
                                rgb_matrix[np.where(is_in)[0], conformation_num, 1] = 1
                            if (blue in comm):
                                is_in = np.isin(residues, comm)
                                rgb_matrix[np.where(is_in)[0], conformation_num, 2] = 1
                                
                                
                                
        ###Write output files
        np.savetxt(file_name_wh_ex+"_node_BC.dat", 
                   np.transpose([np.mean(BC_data_matrix, axis=1), np.divide(np.std(BC_data_matrix, axis=1),
                                                                            np.sqrt(conf_size-1))]))
        if res_by_res_calculation is True:
            for i in range(max_community_size-1):
                np.savetxt(file_name_wh_ex+"_community_"+str(i+2)+"_res_by_res_matrix.dat",
                           np.mean(community_all_data_matrix[:,:,:,i], axis=2), fmt='%1.2f')
        if segment_calculation is True:
            np.savetxt(file_name_wh_ex+"_comm_btw_segments.dat", comm_mat_btw_segment.T, fmt='%1d')
        if rgb_scheme is True:
            rgb_matrix = np.mean(rgb_matrix, axis=1)
            np.savetxt(file_name_wh_ex+"_rgb_data_matrix.dat", rgb_matrix, fmt='%1.1f')
            

@> 2609 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> DCD file contains 211 coordinate sets for 2609 atoms.
@> DCD file was parsed in 0.00 seconds.
@> 6.32 MB parsed at input rate 2094.19 MB/s.
@> 211 coordinate sets parsed at input rate 69960 frame/s.
@> DCD file contains 210 coordinate sets for 2609 atoms.
@> DCD file was parsed in 0.00 seconds.
@> 6.29 MB parsed at input rate 2700.04 MB/s.
@> 210 coordinate sets parsed at input rate 90200 frame/s.


Ensemble l28r_tp (110:210:10) + l28r_tp_2 (110:210:10)
Here is the residue index of the nodes
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160


Index of MD-simulation frame 0
Index of MD-simulation frame 1
Index of MD-simulation frame 2
Index of MD-simulation frame 3
Index of MD-simulation frame 4
Index of MD-simulation frame 5
Index of MD-simulation frame 6
Index of MD-simulation frame 7
Index of MD-simulation frame 8
Index of MD-simulation frame 9
Index of MD-simulation frame 10
Index of MD-simulation frame 1

@> 2604 atoms and 1 coordinate set(s) were parsed in 0.01s.
@> DCD file contains 211 coordinate sets for 2604 atoms.
@> DCD file was parsed in 0.00 seconds.
@> 6.30 MB parsed at input rate 2122.74 MB/s.
@> 211 coordinate sets parsed at input rate 71049 frame/s.
@> DCD file contains 211 coordinate sets for 2604 atoms.
@> DCD file was parsed in 0.00 seconds.
@> 6.30 MB parsed at input rate 3079.53 MB/s.
@> 211 coordinate sets parsed at input rate 103074 frame/s.


Ensemble wt_tp (110:210:10) + wt_tp_2 (110:210:10)
Here is the residue index of the nodes
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160


Index of MD-simulation frame 0
Index of MD-simulation frame 1
Index of MD-simulation frame 2
Index of MD-simulation frame 3
Index of MD-simulation frame 4
Index of MD-simulation frame 5
Index of MD-simulation frame 6
Index of MD-simulation frame 7
Index of MD-simulation frame 8
Index of MD-simulation frame 9
Index of MD-simulation frame 10
Index of MD-simulation frame 11
In

### Here we write the vmd file contains the RGB coloring information of DHFR.

In [13]:
rgb_file = "l28r_tp"
full_mat_tot = np.loadtxt(rgb_file+"_rgb_data_matrix.dat")
full_mat_tot = np.round(full_mat_tot, decimals=1)
full_mat_tot_row_sums = np.sum(full_mat_tot, axis=1)
full_mat_tot = full_mat_tot/full_mat_tot_row_sums[:, np.newaxis];
full_mat_tot[np.isnan(full_mat_tot)] = 0
full_mat_tot = np.round(full_mat_tot, decimals=1)
full_mat_tot_uniq = np.unique(full_mat_tot, axis=0)
residues = np.asarray(residues)
coloria = red+blue+green

f = open(rgb_file+"_color_6.vmd", 'w')
f.write("""
display projection Orthographic
color change rgb  32 1 1 1 
color Display Background 32
axes location Off
display depthcue off
color scale method RGB
\n
""")
f.write("mol new {}.pdb\n".format(str(rgb_file)))
f.write("mol modstyle 0 0 NewCartoon {0.300000 50.000000 4.100000 0}\n")

for i in range(len(full_mat_tot_uniq)):
    f.write("color change rgb  {:d} {:03.1f} {:03.1f} {:03.1f}\n".format(i, full_mat_tot_uniq[i][0], full_mat_tot_uniq[i][1], full_mat_tot_uniq[i][2]))
    f.write("mol addrep 0\n")
    f.write("mol modselect {:d} 0 resid {}\n".format(i+1,' '.join(map(str, residues[np.where(np.all(full_mat_tot==full_mat_tot_uniq[i], axis=1))[0]]))))
    f.write("mol modstyle {:d} 0 NewCartoon {{0.300000 50.000000 4.100000 0}}\n".format(i+1))
    f.write("mol modcolor {:d} 0 ColorID {:d}\n".format(i+1,i))
    
f.write("mol addrep 0\n")
f.write("mol modselect {:d} 0 resname DHF or resname NDPH\n".format(int(len(full_mat_tot_uniq)+1)))
f.write("mol modstyle {:d} 0 Licorice {{0.300000 50.000000 50.000000}}\n".format(int(len(full_mat_tot_uniq)+1)))

f.write("mol addrep 0\n")
f.write("mol modselect {:d} 0 resid {} and name CA\n".format(int(len(full_mat_tot_uniq)+2), red[0]))
f.write("mol modstyle {:d} 0 CPK {{6.000000 0.300000 40.000000 40.000000}}\n".format(int(len(full_mat_tot_uniq)+2)))
f.write("mol modcolor {:d} 0 ColorID {:d}\n".format(int(len(full_mat_tot_uniq)+2),np.where(np.all(full_mat_tot_uniq == full_mat_tot[red[0]], axis=1))[0][0]))

f.write("mol addrep 0\n")
f.write("mol modselect {:d} 0 resid {} and name CA\n".format(int(len(full_mat_tot_uniq)+3), blue[0]))
f.write("mol modstyle {:d} 0 CPK {{6.000000 0.300000 40.000000 40.000000}}\n".format(int(len(full_mat_tot_uniq)+3)))
f.write("mol modcolor {:d} 0 ColorID {:d}\n".format(int(len(full_mat_tot_uniq)+3),np.where(np.all(full_mat_tot_uniq == full_mat_tot[blue[0]], axis=1))[0][0]))

f.write("mol addrep 0\n")
f.write("mol modselect {:d} 0 resid {} and name CA\n".format(int(len(full_mat_tot_uniq)+4), green[0]))
f.write("mol modstyle {:d} 0 CPK {{6.000000 0.300000 40.000000 40.000000}}\n".format(int(len(full_mat_tot_uniq)+4)))
f.write("mol modcolor {:d} 0 ColorID {:d}\n".format(int(len(full_mat_tot_uniq)+4),np.where(np.all(full_mat_tot_uniq == full_mat_tot[green[0]], axis=1))[0][0]))

f.close()


/var/folders/7p/q8xp6f956jgglgh04hjl6dqr0000gn/T/ipykernel_4854/1863606099.py:5: RuntimeWarning: invalid value encountered in divide
  full_mat_tot = full_mat_tot/full_mat_tot_row_sums[:, np.newaxis];
